In [1]:
import numpy as np # linear algebra
from numpy import sort
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import matplotlib.pyplot as plt
%matplotlib inline

from pandas import read_csv

import sklearn

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_selection import SelectFromModel

import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import plot_importance
from xgboost import plot_tree

from numpy import sqrt

from itertools import product

import graphviz

import shap

/Users/karina.vihta/opt/anaconda3/envs/xgb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data preparation/feature engineering

In [2]:
dataset = pd.read_csv('https://github.com/karinadorisvihta/AMR_forecasting/blob/main/Dataset/dataset_sim.csv?raw=true', header=0)
usage = pd.read_csv('https://github.com/karinadorisvihta/AMR_forecasting/blob/main/Dataset/usage_sim.csv?raw=true', header=0)

In [3]:
dataset.head()

,TrustName.x,path_abx,fyear,nons
0,trust_1,E. COLI_AMOXICILLIN/CLAVULANIC ACID,2016-2017,57.651902
1,trust_2,E. COLI_AMOXICILLIN/CLAVULANIC ACID,2016-2017,33.633121
2,trust_3,E. COLI_AMOXICILLIN/CLAVULANIC ACID,2016-2017,34.732463
3,trust_4,E. COLI_AMOXICILLIN/CLAVULANIC ACID,2016-2017,2.002430
4,trust_5,E. COLI_AMOXICILLIN/CLAVULANIC ACID,2016-2017,50.281305


In [4]:
dataset["path_abx"] = dataset["path_abx"] + '-res' 
#print(dataset.head())
#print(dataset.shape)

In [5]:
usage.head()

,TrustName.x,abx,fyear,usage_rate
0,trust_1,Amoxicillin,2014-2015,69.525694
1,trust_2,Amoxicillin,2014-2015,51.876584
2,trust_3,Amoxicillin,2014-2015,65.811825
3,trust_4,Amoxicillin,2014-2015,53.710891
4,trust_5,Amoxicillin,2014-2015,40.302989


In [6]:
usage['pathogen'] = 'usage'
#usage

In [7]:
usage.columns = ['TrustName.x', 'antibiotic', 'fyear', 'nons_per',
       'pathogen']
usage["path_abx"] = usage["pathogen"] + '-' + usage["antibiotic"]
#usage

In [8]:
dataset['nons_per'] = dataset['nons']*1
#dataset['nons_per'] = dataset['nons']*100

In [9]:
dataset_wide=pd.pivot(dataset, index=['TrustName.x','path_abx'], columns = 'fyear',values = 'nons_per') #Reshape from long to wide
dataset_wide['2017-2018_diff'] = dataset_wide['2017-2018']-dataset_wide['2016-2017']
dataset_wide['2018-2019_diff'] = dataset_wide['2018-2019']-dataset_wide['2017-2018']
dataset_wide['2019-2020_diff'] = dataset_wide['2019-2020']-dataset_wide['2018-2019']
dataset_wide['2020-2021_diff'] = dataset_wide['2020-2021']-dataset_wide['2019-2020']
dataset_wide['2021-2022_diff'] = dataset_wide['2021-2022']-dataset_wide['2020-2021']
dataset_wide = dataset_wide.reset_index()
dataset_long = dataset_wide.melt(id_vars=["TrustName.x","path_abx"], value_name = "nons_per")
dataset_wide=pd.pivot(dataset_long, index=['TrustName.x'], columns = ['path_abx','fyear'],values = 'nons_per') 
#Reshape from long to wide
dataset_wide
# 100 examples in total now (across all 27 pathogen-drug combinations)

path_abx,E. COLI_AMOXICILLIN/CLAVULANIC ACID-res,E. COLI_CARBAPENEMS-res,E. COLI_CEPHALOSPORINS-res,E. COLI_CIPROFLOXACIN-res,E. COLI_GENTAMICIN-res,E. COLI_PIPERACILLIN/TAZOBACTAM-res,KLEBSIELLA SP._AMOXICILLIN/CLAVULANIC ACID-res,KLEBSIELLA SP._CARBAPENEMS-res,KLEBSIELLA SP._CEPHALOSPORINS-res,KLEBSIELLA SP._CIPROFLOXACIN-res,...,MSSA_CLARITHROMYCIN-res,MSSA_CLINDAMYCIN-res,MSSA_ERYTHROMYCIN-res,MSSA_TETRACYCLINE-res,MSSA_VANCOMYCIN-res,P. AERUGINOSA_CARBAPENEMS-res,P. AERUGINOSA_CEFTAZIDIME-res,P. AERUGINOSA_CIPROFLOXACIN-res,P. AERUGINOSA_GENTAMICIN-res,P. AERUGINOSA_PIPERACILLIN/TAZOBACTAM-res
fyear,2016-2017,2016-2017,2016-2017,2016-2017,2016-2017,2016-2017,2016-2017,2016-2017,2016-2017,2016-2017,...,2021-2022_diff,2021-2022_diff,2021-2022_diff,2021-2022_diff,2021-2022_diff,2021-2022_diff,2021-2022_diff,2021-2022_diff,2021-2022_diff,2021-2022_diff
TrustName.x,,,,,,,,,,,,,,,,,,,,,
trust_1,57.651902,78.394648,27.077802,65.376668,7.495675,14.415232,27.081543,56.944327,71.315019,7.246915,...,54.995568,17.616243,9.047245,56.247353,0.086784,2.966832,18.195947,-29.280028,24.583997,-25.316361
trust_10,77.033407,53.376487,66.238551,94.958532,42.043490,12.806442,86.098936,28.285041,64.014849,21.666214,...,-6.030510,65.683996,-45.064998,56.639635,22.561801,10.157633,1.615785,-15.991609,-36.952000,21.958507
trust_100,36.330054,58.493656,65.026150,23.526886,56.935269,61.792612,90.794640,7.772155,11.869032,70.307297,...,-50.981686,-13.409849,-32.400030,5.231321,-33.996090,-60.778246,-9.040557,-48.705178,52.446144,42.326587
trust_11,73.518431,96.384333,44.452740,70.903790,58.727812,25.077853,67.355499,64.322156,62.892776,54.799887,...,0.996350,36.283387,-24.557506,29.257525,23.480437,0.184877,-90.823064,16.844484,37.299781,54.471930
trust_12,97.187564,77.459154,62.714618,41.330541,80.669443,33.154699,1.304103,94.811884,27.140900,72.005046,...,60.825150,60.315476,-53.890423,-25.683576,-56.203537,70.423343,14.713659,34.206405,-12.201409,-74.752487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
trust_95,83.525532,12.474209,1.616048,35.054477,20.374381,88.902149,42.590513,71.578096,20.999806,57.758128,...,-27.056014,20.011719,-33.269788,83.107854,32.593651,-24.185507,-48.339853,-30.651703,-49.752433,5.214857
trust_96,14.381704,97.472581,4.770998,84.555612,51.378114,42.445210,50.815808,72.753363,37.584344,24.872636,...,43.258005,-7.145228,57.395992,42.068083,-2.860059,-67.759507,65.569442,-25.335247,-23.066440,1.390617


In [10]:
#dataset_wide.shape

In [11]:
print(dataset_wide.isna().sum().sort_values(ascending=False))
# drop all columns that have only missing values for sure
dataset_wide = dataset_wide.dropna(axis=1, how='all')
print(dataset_wide.shape)

path_abx                                   fyear         
E. COLI_AMOXICILLIN/CLAVULANIC ACID-res    2016-2017         0
E. COLI_PIPERACILLIN/TAZOBACTAM-res        2019-2020_diff    0
P. AERUGINOSA_PIPERACILLIN/TAZOBACTAM-res  2017-2018_diff    0
E. COLI_AMOXICILLIN/CLAVULANIC ACID-res    2018-2019_diff    0
E. COLI_CARBAPENEMS-res                    2018-2019_diff    0
                                                            ..
P. AERUGINOSA_CIPROFLOXACIN-res            2019-2020         0
P. AERUGINOSA_GENTAMICIN-res               2019-2020         0
P. AERUGINOSA_PIPERACILLIN/TAZOBACTAM-res  2019-2020         0
E. COLI_AMOXICILLIN/CLAVULANIC ACID-res    2020-2021         0
P. AERUGINOSA_PIPERACILLIN/TAZOBACTAM-res  2021-2022_diff    0
Length: 242, dtype: int64
(100, 242)


In [12]:
usage_wide=pd.pivot(usage, index=['TrustName.x','path_abx'], columns = 'fyear',values = 'nons_per') #Reshape from long to wide
usage_wide['2015-2016_diff'] = usage_wide['2015-2016']-usage_wide['2014-2015']
usage_wide['2016-2017_diff'] = usage_wide['2016-2017']-usage_wide['2015-2016']
usage_wide['2017-2018_diff'] = usage_wide['2017-2018']-usage_wide['2016-2017']
usage_wide['2018-2019_diff'] = usage_wide['2018-2019']-usage_wide['2017-2018']
usage_wide['2019-2020_diff'] = usage_wide['2019-2020']-usage_wide['2018-2019']
usage_wide['2020-2021_diff'] = usage_wide['2020-2021']-usage_wide['2019-2020']
usage_wide = usage_wide.reset_index()
usage_long = usage_wide.melt(id_vars=["TrustName.x","path_abx"], value_name = "nons_per")
usage_wide=pd.pivot(usage_long, index=['TrustName.x'], columns = ['path_abx','fyear'],values = 'nons_per') 
#Reshape from long to wide
usage_wide

path_abx,usage-Amoxicillin,usage-Amoxicillin/clavulanic acid,usage-Azithromycin,usage-Cefalexin,usage-Cefotaxime,usage-Ceftazidime,usage-Ceftriaxone,usage-Cefuroxime,usage-Ciprofloxacin,usage-Clarithromycin,...,usage-Ofloxacin,usage-Penicillin g,usage-Penicillin v,usage-Piperacillin/tazobactam,usage-Pivmecillinam,usage-Sulfamethoxazole/trimethoprim,usage-Teicoplanin,usage-Tobramycin,usage-Trimethoprim,usage-Vancomycin_iv
fyear,2014-2015,2014-2015,2014-2015,2014-2015,2014-2015,2014-2015,2014-2015,2014-2015,2014-2015,2014-2015,...,2020-2021_diff,2020-2021_diff,2020-2021_diff,2020-2021_diff,2020-2021_diff,2020-2021_diff,2020-2021_diff,2020-2021_diff,2020-2021_diff,2020-2021_diff
TrustName.x,,,,,,,,,,,,,,,,,,,,,
trust_1,69.525694,23.453881,12.189666,12.246378,34.280933,61.240498,48.982088,99.763441,65.600226,18.873458,...,-25.528779,16.813382,-13.092132,10.989088,-52.085692,-5.621480,27.161941,-45.384719,0.566404,-69.376262
trust_10,60.904397,25.506442,29.065922,59.219749,67.585762,56.075551,10.933414,89.431911,62.437783,83.326475,...,-10.786954,40.161686,-56.568781,-2.010473,61.963622,-16.858871,3.306233,33.632555,37.927269,28.643326
trust_100,90.469623,33.202778,99.865338,14.415564,63.887136,92.207377,66.491147,71.006304,5.646104,40.294156,...,26.935626,-9.241617,-45.080195,24.261190,60.872590,-21.177128,84.211946,-4.069498,-88.368390,7.856175
trust_11,79.309167,97.172550,10.758600,28.242865,51.705812,11.102631,35.747417,69.237207,86.204443,99.961509,...,-56.985411,-13.564476,-33.977532,-33.474332,-34.450101,-36.511966,47.050778,7.339091,26.746719,-12.073473
trust_12,94.853821,11.046690,18.092243,42.170572,45.522822,95.508219,6.316687,76.630424,20.801676,46.056694,...,-6.963759,0.417843,38.350081,-34.162621,23.764525,33.659131,-35.127313,22.393364,-11.875203,9.287533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
trust_95,90.496888,56.947968,1.961173,71.992655,92.741684,39.510290,61.937579,54.999523,77.700052,92.817098,...,-27.622084,-49.073743,45.553555,-47.300674,-65.442701,43.866164,-3.705926,-9.144343,58.929230,-52.026359
trust_96,48.838440,18.287884,3.951025,42.976893,89.826615,65.394331,41.663643,92.817324,82.587918,72.242488,...,-79.343176,13.361390,61.925840,-8.696114,27.765151,-17.387097,32.209929,-16.471057,-47.427228,-56.642602


In [ ]:
#usage_wide.isna().sum().sort_values(ascending=False)

In [13]:
res_and_usage = pd.merge(dataset_wide, usage_wide, left_index=True, right_index=True)
res_and_usage

path_abx,E. COLI_AMOXICILLIN/CLAVULANIC ACID-res,E. COLI_CARBAPENEMS-res,E. COLI_CEPHALOSPORINS-res,E. COLI_CIPROFLOXACIN-res,E. COLI_GENTAMICIN-res,E. COLI_PIPERACILLIN/TAZOBACTAM-res,KLEBSIELLA SP._AMOXICILLIN/CLAVULANIC ACID-res,KLEBSIELLA SP._CARBAPENEMS-res,KLEBSIELLA SP._CEPHALOSPORINS-res,KLEBSIELLA SP._CIPROFLOXACIN-res,...,usage-Ofloxacin,usage-Penicillin g,usage-Penicillin v,usage-Piperacillin/tazobactam,usage-Pivmecillinam,usage-Sulfamethoxazole/trimethoprim,usage-Teicoplanin,usage-Tobramycin,usage-Trimethoprim,usage-Vancomycin_iv
fyear,2016-2017,2016-2017,2016-2017,2016-2017,2016-2017,2016-2017,2016-2017,2016-2017,2016-2017,2016-2017,...,2020-2021_diff,2020-2021_diff,2020-2021_diff,2020-2021_diff,2020-2021_diff,2020-2021_diff,2020-2021_diff,2020-2021_diff,2020-2021_diff,2020-2021_diff
TrustName.x,,,,,,,,,,,,,,,,,,,,,
trust_1,57.651902,78.394648,27.077802,65.376668,7.495675,14.415232,27.081543,56.944327,71.315019,7.246915,...,-25.528779,16.813382,-13.092132,10.989088,-52.085692,-5.621480,27.161941,-45.384719,0.566404,-69.376262
trust_10,77.033407,53.376487,66.238551,94.958532,42.043490,12.806442,86.098936,28.285041,64.014849,21.666214,...,-10.786954,40.161686,-56.568781,-2.010473,61.963622,-16.858871,3.306233,33.632555,37.927269,28.643326
trust_100,36.330054,58.493656,65.026150,23.526886,56.935269,61.792612,90.794640,7.772155,11.869032,70.307297,...,26.935626,-9.241617,-45.080195,24.261190,60.872590,-21.177128,84.211946,-4.069498,-88.368390,7.856175
trust_11,73.518431,96.384333,44.452740,70.903790,58.727812,25.077853,67.355499,64.322156,62.892776,54.799887,...,-56.985411,-13.564476,-33.977532,-33.474332,-34.450101,-36.511966,47.050778,7.339091,26.746719,-12.073473
trust_12,97.187564,77.459154,62.714618,41.330541,80.669443,33.154699,1.304103,94.811884,27.140900,72.005046,...,-6.963759,0.417843,38.350081,-34.162621,23.764525,33.659131,-35.127313,22.393364,-11.875203,9.287533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
trust_95,83.525532,12.474209,1.616048,35.054477,20.374381,88.902149,42.590513,71.578096,20.999806,57.758128,...,-27.622084,-49.073743,45.553555,-47.300674,-65.442701,43.866164,-3.705926,-9.144343,58.929230,-52.026359
trust_96,14.381704,97.472581,4.770998,84.555612,51.378114,42.445210,50.815808,72.753363,37.584344,24.872636,...,-79.343176,13.361390,61.925840,-8.696114,27.765151,-17.387097,32.209929,-16.471057,-47.427228,-56.642602


In [14]:
path_drug = dataset_wide.columns.get_level_values(0).unique()
N_r=len(path_drug)
usage_abx = usage_wide.columns.get_level_values(0).unique()
N_u=len(usage_abx)

In [15]:
# using all previous usage and resistance
featureeng = "res4usage6defaultparam"
for p in range(22):
    dataset_wide=res_and_usage
    
    dataset_2021_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2020-2021')])
    d1 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2019-2020']
    d2 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2018-2019']
    d3 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2017-2018']
    d4 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2016-2017']
    d5 = dataset_2021_selabx_nona.iloc[:, np.logical_and(dataset_2021_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2021_selabx_nona.columns.get_level_values(1)=='2015-2016')]
    d6 = dataset_2021_selabx_nona.iloc[:, np.logical_and(dataset_2021_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2021_selabx_nona.columns.get_level_values(1)=='2014-2015')]
    X_2021 = pd.merge(d1,d2,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d3,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d4,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d5,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d6,left_index=True,right_index=True)
    #X_2021["noise"] = np.random.normal(loc = 0, scale = 1, size = X_2021.shape[0])
    y_2021 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns==(path_drug[p], '2020-2021')]

    dataset_2022_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2021-2022')])
    d1 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2020-2021']
    d2 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2019-2020']
    d3 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2018-2019']
    d4 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2017-2018']
    d5 = dataset_2022_selabx_nona.iloc[:, np.logical_and(dataset_2022_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2022_selabx_nona.columns.get_level_values(1)=='2016-2017')]
    d6 = dataset_2022_selabx_nona.iloc[:, np.logical_and(dataset_2022_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2022_selabx_nona.columns.get_level_values(1)=='2015-2016')]
    X_2022 = pd.merge(d1,d2,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d3,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d4,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d5,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d6,left_index=True,right_index=True)
    #X_2022["noise"] = np.random.normal(loc = 0, scale = 1, size = X_2022.shape[0])
    y_2022 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns==(path_drug[p], '2021-2022')]

    ####
    
    X_2021 = X_2021.rename(columns={'2014-2015': 'prev6', '2015-2016': 'prev5', '2016-2017': 'prev4', '2017-2018': 'prev3', '2018-2019': 'prev2', '2019-2020': 'prev1'})
    X_2022 = X_2022.rename(columns={'2015-2016': 'prev6', '2016-2017': 'prev5', '2017-2018': 'prev4', '2018-2019': 'prev3', '2019-2020': 'prev2', '2020-2021': 'prev1'})

    columnsinboth = X_2021.columns.intersection(X_2022.columns) # what is in both
    X_2021 = X_2021[X_2021.columns.intersection(columnsinboth)]
    X_2022 = X_2022[X_2022.columns.intersection(columnsinboth)]

    X_2021_np=np.array(X_2021)
    y_2021_np=np.array(y_2021)
    X_2022_np=np.array(X_2022)
    y_2022_np=np.array(y_2022)

    ####
    model = XGBRegressor(
                         objective='reg:absoluteerror',
                         eval_metric=mean_absolute_error, 
                         seed=42)

    fit = model.fit(
        X_2021_np, 
        y_2021_np, 
        verbose=True)
 
    pred_2021 = fit.predict(X_2021_np) 
    pred_2022 = fit.predict(X_2022_np)
    
    #pd.DataFrame(X_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_X_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(X_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_X_2022_"}{featureeng}{".csv"}')
    
    #pd.DataFrame(y_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_y_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(y_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_y_2022_"}{featureeng}{".csv"}')
    
    #pd.DataFrame(pred_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_pred_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(pred_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_pred_2022_"}{featureeng}{".csv"}')

In [16]:
# using all previous usage and resistance - looking at feature importance
featureeng = "res4usage6defaultparam"
for p in range(22):

    dataset_wide=res_and_usage
    
    dataset_2021_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2020-2021')])
    d1 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2019-2020']
    d2 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2018-2019']
    d3 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2017-2018']
    d4 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2016-2017']
    d5 = dataset_2021_selabx_nona.iloc[:, np.logical_and(dataset_2021_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2021_selabx_nona.columns.get_level_values(1)=='2015-2016')]
    d6 = dataset_2021_selabx_nona.iloc[:, np.logical_and(dataset_2021_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2021_selabx_nona.columns.get_level_values(1)=='2014-2015')]
    X_2021 = pd.merge(d1,d2,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d3,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d4,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d5,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d6,left_index=True,right_index=True)
    #X_2021["noise"] = np.random.normal(loc = 0, scale = 1, size = X_2021.shape[0])
    y_2021 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns==(path_drug[p], '2020-2021')]

    dataset_2022_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2021-2022')])
    d1 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2020-2021']
    d2 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2019-2020']
    d3 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2018-2019']
    d4 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2017-2018']
    d5 = dataset_2022_selabx_nona.iloc[:, np.logical_and(dataset_2022_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2022_selabx_nona.columns.get_level_values(1)=='2016-2017')]
    d6 = dataset_2022_selabx_nona.iloc[:, np.logical_and(dataset_2022_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2022_selabx_nona.columns.get_level_values(1)=='2015-2016')]
    X_2022 = pd.merge(d1,d2,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d3,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d4,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d5,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d6,left_index=True,right_index=True)
    #X_2022["noise"] = np.random.normal(loc = 0, scale = 1, size = X_2022.shape[0])
    y_2022 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns==(path_drug[p], '2021-2022')]

    ####
    
    X_2021 = X_2021.rename(columns={'2014-2015': 'prev6', '2015-2016': 'prev5', '2016-2017': 'prev4', '2017-2018': 'prev3', '2018-2019': 'prev2', '2019-2020': 'prev1'})
    X_2022 = X_2022.rename(columns={'2015-2016': 'prev6', '2016-2017': 'prev5', '2017-2018': 'prev4', '2018-2019': 'prev3', '2019-2020': 'prev2', '2020-2021': 'prev1'})

    columnsinboth = X_2021.columns.intersection(X_2022.columns) # what is in both
    X_2021 = X_2021[X_2021.columns.intersection(columnsinboth)]
    X_2022 = X_2022[X_2022.columns.intersection(columnsinboth)]

    X_2021_np=np.array(X_2021)
    y_2021_np=np.array(y_2021)
    X_2022_np=np.array(X_2022)
    y_2022_np=np.array(y_2022)

    ####
    model = XGBRegressor(
                         objective='reg:absoluteerror',
                         eval_metric=mean_absolute_error, 
                         seed=42)

    fit = model.fit(
        X_2021_np, 
        y_2021_np, 
        verbose=True)
 
    pred_2021 = fit.predict(X_2021_np) 
    pred_2022 = fit.predict(X_2022_np)
    
    print(path_drug[p])
    # DF, based on which importance is checked
    X_importance = X_2022_np
    # Explain model predictions using shap library:
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_importance)
    # Plot summary_plot as barplot:
    #shap.summary_plot(shap_values, X_importance, plot_type='bar')
    vals= np.abs(shap_values).mean(0) 
    feature_importance = pd.DataFrame(list(zip(X_2022.columns,vals)),columns=['col_name','feature_importance_vals']) 
    feature_importance.sort_values(by=['feature_importance_vals'],ascending=False,inplace=True) 
    print(feature_importance.head())
    #pd.DataFrame(feature_importance).to_csv(f'{path_drug[p].replace("/","_")}{"_feature_importance_"}{featureeng}{".csv"}')

E. COLI_AMOXICILLIN/CLAVULANIC ACID-res
                                         col_name  feature_importance_vals
186        (P. AERUGINOSA_CEFTAZIDIME-res, prev4)                 2.381780
30                   (usage-Ciprofloxacin, prev1)                 1.779142
273                 (usage-Flucloxacillin, prev6)                 1.488964
5    (E. COLI_PIPERACILLIN/TAZOBACTAM-res, prev1)                 1.331851
110                   (usage-Trimethoprim, prev2)                 0.918879


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


E. COLI_CARBAPENEMS-res
                                  col_name  feature_importance_vals
248               (usage-Ofloxacin, prev5)                 2.437565
132  (P. AERUGINOSA_GENTAMICIN-res, prev3)                 2.159345
139             (usage-Ceftazidime, prev3)                 1.726988
148            (usage-Erythromycin, prev3)                 1.721837
20   (P. AERUGINOSA_GENTAMICIN-res, prev1)                 1.338091


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


E. COLI_CEPHALOSPORINS-res
                                   col_name  feature_importance_vals
136             (usage-Azithromycin, prev3)                 3.687881
160             (usage-Penicillin v, prev3)                 3.148523
70           (MSSA_ERYTHROMYCIN-res, prev2)                 3.084765
217  (usage-Piperacillin/tazobactam, prev4)                 1.474036
208                (usage-Linezolid, prev4)                 1.196983


...

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


P. AERUGINOSA_GENTAMICIN-res
                                      col_name  feature_importance_vals
121  (KLEBSIELLA SP._CIPROFLOXACIN-res, prev3)                 1.793017
53                   (usage-Tobramycin, prev1)                 1.364233
18      (P. AERUGINOSA_CEFTAZIDIME-res, prev1)                 1.284520
284                (usage-Penicillin v, prev6)                 1.159184
209                 (usage-Lymecycline, prev4)                 1.050853


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


P. AERUGINOSA_PIPERACILLIN/TAZOBACTAM-res
                                              col_name  \
231                          (usage-Cefuroxime, prev5)   
38                           (usage-Gentamicin, prev1)   
67   (KLEBSIELLA SP._PIPERACILLIN/TAZOBACTAM-res, p...   
250                        (usage-Penicillin v, prev5)   
230                         (usage-Ceftriaxone, prev5)   

     feature_importance_vals  
231                 6.608608  
38                  2.683474  
67                  0.891138  
250                 0.883672  
230                 0.876430  


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


In [17]:
# summarising feature importance only in those trusts with changes larger than 10% in resistance from the previous year
featureeng = "res4usage6defaultparam"
for p in range(22):

    dataset_wide=res_and_usage
    
    dataset_2021_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2020-2021')])
    d1 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2019-2020']
    d2 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2018-2019']
    d3 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2017-2018']
    d4 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2016-2017']
    d5 = dataset_2021_selabx_nona.iloc[:, np.logical_and(dataset_2021_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2021_selabx_nona.columns.get_level_values(1)=='2015-2016')]
    d6 = dataset_2021_selabx_nona.iloc[:, np.logical_and(dataset_2021_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2021_selabx_nona.columns.get_level_values(1)=='2014-2015')]
    X_2021 = pd.merge(d1,d2,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d3,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d4,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d5,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d6,left_index=True,right_index=True)
    #X_2021["noise"] = np.random.normal(loc = 0, scale = 1, size = X_2021.shape[0])
    y_2021 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns==(path_drug[p], '2020-2021')]

    dataset_2022_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2021-2022')])
    d1 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2020-2021']
    d2 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2019-2020']
    d3 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2018-2019']
    d4 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2017-2018']
    d5 = dataset_2022_selabx_nona.iloc[:, np.logical_and(dataset_2022_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2022_selabx_nona.columns.get_level_values(1)=='2016-2017')]
    d6 = dataset_2022_selabx_nona.iloc[:, np.logical_and(dataset_2022_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2022_selabx_nona.columns.get_level_values(1)=='2015-2016')]
    X_2022 = pd.merge(d1,d2,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d3,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d4,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d5,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d6,left_index=True,right_index=True)
    #X_2022["noise"] = np.random.normal(loc = 0, scale = 1, size = X_2022.shape[0])
    y_2022 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns==(path_drug[p], '2021-2022')]

    ####
    
    X_2021 = X_2021.rename(columns={'2014-2015': 'prev6', '2015-2016': 'prev5', '2016-2017': 'prev4', '2017-2018': 'prev3', '2018-2019': 'prev2', '2019-2020': 'prev1'})
    X_2022 = X_2022.rename(columns={'2015-2016': 'prev6', '2016-2017': 'prev5', '2017-2018': 'prev4', '2018-2019': 'prev3', '2019-2020': 'prev2', '2020-2021': 'prev1'})

    columnsinboth = X_2021.columns.intersection(X_2022.columns) # what is in both
    X_2021 = X_2021[X_2021.columns.intersection(columnsinboth)]
    X_2022 = X_2022[X_2022.columns.intersection(columnsinboth)]

    X_2021_np=np.array(X_2021)
    y_2021_np=np.array(y_2021)
    X_2022_np=np.array(X_2022)
    y_2022_np=np.array(y_2022)

    ####
    model = XGBRegressor(
                         objective='reg:absoluteerror',
                         eval_metric=mean_absolute_error, 
                         seed=42)

    fit = model.fit(
        X_2021_np, 
        y_2021_np, 
        verbose=True)
 
    pred_2021 = fit.predict(X_2021_np) 
    pred_2022 = fit.predict(X_2022_np)
    
    print(path_drug[p])
    # DF, based on which importance is checked
    X_importance = X_2022_np
    # Explain model predictions using shap library:
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_importance)
    # Plot summary_plot as barplot:
    #shap.summary_plot(shap_values, X_importance, plot_type='bar')
    vals= np.abs(shap_values[list(list(np.where(abs(np.subtract(y_2022_np, pred_2022))>10))[0])]).mean(0) 
    feature_importance = pd.DataFrame(list(zip(X_2022.columns,vals)),columns=['col_name','feature_importance_vals']) 
    feature_importance.sort_values(by=['feature_importance_vals'],ascending=False,inplace=True) 
    #print(feature_importance.head())
    #pd.DataFrame(feature_importance).to_csv(f'{path_drug[p].replace("/","_")}{"_feature_importance_largechanges_"}{featureeng}{".csv"}')

E. COLI_AMOXICILLIN/CLAVULANIC ACID-res


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


E. COLI_CARBAPENEMS-res


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


E. COLI_CEPHALOSPORINS-res


...

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


P. AERUGINOSA_GENTAMICIN-res


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


P. AERUGINOSA_PIPERACILLIN/TAZOBACTAM-res


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


In [18]:
# tuning the number of estimators, the maximum depth and the minimum child weight in order to avoid overfitting and improve generalization
featureeng = "res4usage6tunparam"
for p in range(22):
    dataset_wide=res_and_usage
    
    dataset_2021_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2020-2021')])
    d1 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2019-2020']
    d2 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2018-2019']
    d3 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2017-2018']
    d4 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2016-2017']
    d5 = dataset_2021_selabx_nona.iloc[:, np.logical_and(dataset_2021_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2021_selabx_nona.columns.get_level_values(1)=='2015-2016')]
    d6 = dataset_2021_selabx_nona.iloc[:, np.logical_and(dataset_2021_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2021_selabx_nona.columns.get_level_values(1)=='2014-2015')]
    X_2021 = pd.merge(d1,d2,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d3,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d4,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d5,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d6,left_index=True,right_index=True)
    #X_2021["noise"] = np.random.normal(loc = 0, scale = 1, size = X_2021.shape[0])
    y_2021 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns==(path_drug[p], '2020-2021')]

    dataset_2022_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2021-2022')])
    d1 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2020-2021']
    d2 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2019-2020']
    d3 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2018-2019']
    d4 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2017-2018']
    d5 = dataset_2022_selabx_nona.iloc[:, np.logical_and(dataset_2022_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2022_selabx_nona.columns.get_level_values(1)=='2016-2017')]
    d6 = dataset_2022_selabx_nona.iloc[:, np.logical_and(dataset_2022_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2022_selabx_nona.columns.get_level_values(1)=='2015-2016')]
    X_2022 = pd.merge(d1,d2,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d3,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d4,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d5,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d6,left_index=True,right_index=True)
    #X_2022["noise"] = np.random.normal(loc = 0, scale = 1, size = X_2022.shape[0])
    y_2022 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns==(path_drug[p], '2021-2022')]

    ####
    
    X_2021 = X_2021.rename(columns={'2014-2015': 'prev6', '2015-2016': 'prev5', '2016-2017': 'prev4', '2017-2018': 'prev3', '2018-2019': 'prev2', '2019-2020': 'prev1'})
    X_2022 = X_2022.rename(columns={'2015-2016': 'prev6', '2016-2017': 'prev5', '2017-2018': 'prev4', '2018-2019': 'prev3', '2019-2020': 'prev2', '2020-2021': 'prev1'})

    columnsinboth = X_2021.columns.intersection(X_2022.columns) # what is in both
    X_2021 = X_2021[X_2021.columns.intersection(columnsinboth)]
    X_2022 = X_2022[X_2022.columns.intersection(columnsinboth)]

    X_2021_np=np.array(X_2021)
    y_2021_np=np.array(y_2021)
    X_2022_np=np.array(X_2022)
    y_2022_np=np.array(y_2022)

    ####
    n_estimators = [10, 50, 100]
    max_depth = [1, 3, 5]
    min_child_weight = [3,5,7]
    #n_estimators = [10, 50, 100, 150, 200, 300, 400]
    #max_depth = [1, 2, 3, 4, 5, 8, 12]
    #min_child_weight = [3,5,7,10]
    #eta = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5]
    
    # look at which ones got chosen for the different pathogen-antibiotic combinations
    # 
    param_grid = dict(n_estimators=n_estimators, max_depth=max_depth, min_child_weight=min_child_weight)
    kfold = KFold(n_splits=3, shuffle=True, random_state=7)
    model = XGBRegressor(objective='reg:absoluteerror', seed=42)
    grid_search = GridSearchCV(model, param_grid, scoring="neg_mean_absolute_error", n_jobs=-1, cv=kfold, verbose=1)
    grid_result = grid_search.fit(X_2021_np, y_2021_np)
     # summarize results
    print(path_drug[p])
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
     print("%f (%f) with: %r" % (mean, stdev, param))
    
    model_t = XGBRegressor(**grid_search.best_params_,eval_metric=mean_absolute_error,seed=42)
    fit = model_t.fit(
        X_2021_np, 
        y_2021_np, 
        #eval_set=[(X_2021_np, y_2021_np), (X_2022_np, y_2022_np)], 
        verbose=True)
 
    pred_2021 = fit.predict(X_2021_np) 
    pred_2022 = fit.predict(X_2022_np)
    
    #pd.DataFrame(X_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_X_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(X_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_X_2022_"}{featureeng}{".csv"}')
    
    #pd.DataFrame(y_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_y_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(y_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_y_2022_"}{featureeng}{".csv"}')
    
    #pd.DataFrame(pred_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_pred_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(pred_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_pred_2022_"}{featureeng}{".csv"}')

Fitting 3 folds for each of 27 candidates, totalling 81 fits
E. COLI_AMOXICILLIN/CLAVULANIC ACID-res
Best: -25.637150 using {'max_depth': 5, 'min_child_weight': 3, 'n_estimators': 100}
-25.898488 (1.203958) with: {'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 10}
-26.016120 (1.122162) with: {'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 50}
-26.122018 (1.018735) with: {'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 100}
-25.898488 (1.203958) with: {'max_depth': 1, 'min_child_weight': 5, 'n_estimators': 10}
-26.016120 (1.122162) with: {'max_depth': 1, 'min_child_weight': 5, 'n_estimators': 50}
-26.122018 (1.018735) with: {'max_depth': 1, 'min_child_weight': 5, 'n_estimators': 100}
-25.898488 (1.203958) with: {'max_depth': 1, 'min_child_weight': 7, 'n_estimators': 10}
-26.016120 (1.122162) with: {'max_depth': 1, 'min_child_weight': 7, 'n_estimators': 50}
-26.122018 (1.018735) with: {'max_depth': 1, 'min_child_weight': 7, 'n_estimators': 100}
-25.967730 (1.176246

E. COLI_GENTAMICIN-res
Best: -24.208991 using {'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 10}
-24.208991 (1.671841) with: {'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 10}
-24.482080 (2.378150) with: {'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 50}
-24.854714 (2.440302) with: {'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 100}
-24.208991 (1.671841) with: {'max_depth': 1, 'min_child_weight': 5, 'n_estimators': 10}
-24.482080 (2.378150) with: {'max_depth': 1, 'min_child_weight': 5, 'n_estimators': 50}
-24.854714 (2.440302) with: {'max_depth': 1, 'min_child_weight': 5, 'n_estimators': 100}
-24.208991 (1.671841) with: {'max_depth': 1, 'min_child_weight': 7, 'n_estimators': 10}
-24.482080 (2.378150) with: {'max_depth': 1, 'min_child_weight': 7, 'n_estimators': 50}
-24.854714 (2.440302) with: {'max_depth': 1, 'min_child_weight': 7, 'n_estimators': 100}
-24.402948 (1.651004) with: {'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 10}
-25.327647 

KLEBSIELLA SP._CEPHALOSPORINS-res
Best: -29.129559 using {'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 10}
-29.220994 (3.559704) with: {'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 10}
-29.280799 (3.551545) with: {'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 50}
-29.198891 (3.761525) with: {'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 100}
-29.220994 (3.559704) with: {'max_depth': 1, 'min_child_weight': 5, 'n_estimators': 10}
-29.280799 (3.551545) with: {'max_depth': 1, 'min_child_weight': 5, 'n_estimators': 50}
-29.198891 (3.761525) with: {'max_depth': 1, 'min_child_weight': 5, 'n_estimators': 100}
-29.220994 (3.559704) with: {'max_depth': 1, 'min_child_weight': 7, 'n_estimators': 10}
-29.280799 (3.551545) with: {'max_depth': 1, 'min_child_weight': 7, 'n_estimators': 50}
-29.198891 (3.761525) with: {'max_depth': 1, 'min_child_weight': 7, 'n_estimators': 100}
-29.226691 (3.557049) with: {'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 10}


MSSA_CLARITHROMYCIN-res
Best: -23.780828 using {'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 50}
-23.951478 (1.125868) with: {'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 10}
-23.780828 (1.125918) with: {'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 50}
-24.017111 (1.004619) with: {'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 100}
-23.951478 (1.125868) with: {'max_depth': 1, 'min_child_weight': 5, 'n_estimators': 10}
-23.780828 (1.125918) with: {'max_depth': 1, 'min_child_weight': 5, 'n_estimators': 50}
-24.017111 (1.004619) with: {'max_depth': 1, 'min_child_weight': 5, 'n_estimators': 100}
-23.951478 (1.125868) with: {'max_depth': 1, 'min_child_weight': 7, 'n_estimators': 10}
-23.780828 (1.125918) with: {'max_depth': 1, 'min_child_weight': 7, 'n_estimators': 50}
-24.017111 (1.004619) with: {'max_depth': 1, 'min_child_weight': 7, 'n_estimators': 100}
-24.073588 (1.327515) with: {'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 10}
-24.042799

Fitting 3 folds for each of 27 candidates, totalling 81 fits
MSSA_VANCOMYCIN-res
Best: -24.281785 using {'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 10}
-24.281785 (1.401664) with: {'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 10}
-24.708288 (1.660510) with: {'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 50}
-24.810752 (2.081384) with: {'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 100}
-24.281785 (1.401664) with: {'max_depth': 1, 'min_child_weight': 5, 'n_estimators': 10}
-24.708288 (1.660510) with: {'max_depth': 1, 'min_child_weight': 5, 'n_estimators': 50}
-24.810752 (2.081384) with: {'max_depth': 1, 'min_child_weight': 5, 'n_estimators': 100}
-24.281785 (1.401664) with: {'max_depth': 1, 'min_child_weight': 7, 'n_estimators': 10}
-24.708288 (1.660510) with: {'max_depth': 1, 'min_child_weight': 7, 'n_estimators': 50}
-24.810752 (2.081384) with: {'max_depth': 1, 'min_child_weight': 7, 'n_estimators': 100}
-24.497107 (1.390360) with: {'max_depth':

P. AERUGINOSA_GENTAMICIN-res
Best: -25.659967 using {'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 10}
-25.659967 (2.793203) with: {'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 10}
-26.139637 (2.497366) with: {'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 50}
-26.332687 (2.303930) with: {'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 100}
-25.659967 (2.793203) with: {'max_depth': 1, 'min_child_weight': 5, 'n_estimators': 10}
-26.139637 (2.497366) with: {'max_depth': 1, 'min_child_weight': 5, 'n_estimators': 50}
-26.332687 (2.303930) with: {'max_depth': 1, 'min_child_weight': 5, 'n_estimators': 100}
-25.659967 (2.793203) with: {'max_depth': 1, 'min_child_weight': 7, 'n_estimators': 10}
-26.139637 (2.497366) with: {'max_depth': 1, 'min_child_weight': 7, 'n_estimators': 50}
-26.332687 (2.303930) with: {'max_depth': 1, 'min_child_weight': 7, 'n_estimators': 100}
-25.704256 (2.875842) with: {'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 10}
-26.6

In [19]:
# using previous usage alone as predictor (all available history), without previous resistance
featureeng = "res0usage6defaultparam"
for p in range(22):
    dataset_wide=res_and_usage
    
    dataset_2021_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2020-2021')])
    #d1 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2019-2020']
    d1 = dataset_2021_selabx_nona.iloc[:, np.logical_and(dataset_2021_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2021_selabx_nona.columns.get_level_values(1)=='2019-2020')]
    #d2 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2018-2019']
    d2 = dataset_2021_selabx_nona.iloc[:, np.logical_and(dataset_2021_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2021_selabx_nona.columns.get_level_values(1)=='2018-2019')]
    #d3 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2017-2018']
    d3 = dataset_2021_selabx_nona.iloc[:, np.logical_and(dataset_2021_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2021_selabx_nona.columns.get_level_values(1)=='2017-2018')]
    #d4 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2016-2017']
    d4 = dataset_2021_selabx_nona.iloc[:, np.logical_and(dataset_2021_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2021_selabx_nona.columns.get_level_values(1)=='2016-2017')]
    d5 = dataset_2021_selabx_nona.iloc[:, np.logical_and(dataset_2021_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2021_selabx_nona.columns.get_level_values(1)=='2015-2016')]
    d6 = dataset_2021_selabx_nona.iloc[:, np.logical_and(dataset_2021_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2021_selabx_nona.columns.get_level_values(1)=='2014-2015')]
    X_2021 = pd.merge(d1,d2,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d3,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d4,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d5,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d6,left_index=True,right_index=True)
    #X_2021["noise"] = np.random.normal(loc = 0, scale = 1, size = X_2021.shape[0])
    y_2021 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns==(path_drug[p], '2020-2021')]

    dataset_2022_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2021-2022')])
    #d1 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2020-2021']
    d1 = dataset_2022_selabx_nona.iloc[:, np.logical_and(dataset_2022_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2022_selabx_nona.columns.get_level_values(1)=='2020-2021')]
    #d2 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2019-2020']
    d2 = dataset_2022_selabx_nona.iloc[:, np.logical_and(dataset_2022_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2022_selabx_nona.columns.get_level_values(1)=='2019-2020')]
    #d3 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2018-2019']
    d3 = dataset_2022_selabx_nona.iloc[:, np.logical_and(dataset_2022_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2022_selabx_nona.columns.get_level_values(1)=='2018-2019')]
    #d4 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2017-2018']
    d4 = dataset_2022_selabx_nona.iloc[:, np.logical_and(dataset_2022_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2022_selabx_nona.columns.get_level_values(1)=='2017-2018')]
    d5 = dataset_2022_selabx_nona.iloc[:, np.logical_and(dataset_2022_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2022_selabx_nona.columns.get_level_values(1)=='2016-2017')]
    d6 = dataset_2022_selabx_nona.iloc[:, np.logical_and(dataset_2022_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2022_selabx_nona.columns.get_level_values(1)=='2015-2016')]
    X_2022 = pd.merge(d1,d2,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d3,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d4,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d5,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d6,left_index=True,right_index=True)
    #X_2022["noise"] = np.random.normal(loc = 0, scale = 1, size = X_2022.shape[0])
    y_2022 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns==(path_drug[p], '2021-2022')]

    ####
    
    X_2021 = X_2021.rename(columns={'2014-2015': 'prev6', '2015-2016': 'prev5', '2016-2017': 'prev4', '2017-2018': 'prev3', '2018-2019': 'prev2', '2019-2020': 'prev1'})
    X_2022 = X_2022.rename(columns={'2015-2016': 'prev6', '2016-2017': 'prev5', '2017-2018': 'prev4', '2018-2019': 'prev3', '2019-2020': 'prev2', '2020-2021': 'prev1'})

    columnsinboth = X_2021.columns.intersection(X_2022.columns) # what is in both
    X_2021 = X_2021[X_2021.columns.intersection(columnsinboth)]
    X_2022 = X_2022[X_2022.columns.intersection(columnsinboth)]

    X_2021_np=np.array(X_2021)
    y_2021_np=np.array(y_2021)
    X_2022_np=np.array(X_2022)
    y_2022_np=np.array(y_2022)

    ####
    model = XGBRegressor(
                         #objective='reg:squarederror',
                         #objective='reg:squaredlogerror',
                         objective='reg:absoluteerror',
                         #objective='reg:pseudohubererror', # needs tuning of delta - somewhere between mae and rmse
                         eval_metric=mean_absolute_error, 
                         #early_stopping_rounds = 10,
                         seed=42)

    fit = model.fit(
        X_2021_np, 
        y_2021_np, 
        #eval_metric="mae", 
        #eval_set=[(X_2021_np, y_2021_np), (X_2022_np, y_2022_np)], 
        #early_stopping_rounds = 10, # this seems to help with generalisation, higher training rmse, but lower testing rmse
        # but then need a separate evaluation set as it uses information from this to tune model
        verbose=True)
    #results = model.evals_result()
    #print(min(results['validation_1']['mean_absolute_error']))
    #best = model.best_ntree_limit
    #print(results['validation_0']['mean_absolute_error'][best-1])
    
    pred_2021 = fit.predict(X_2021_np) 
    pred_2022 = fit.predict(X_2022_np)
    
    #pd.DataFrame(X_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_X_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(X_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_X_2022_"}{featureeng}{".csv"}')
    
    #pd.DataFrame(y_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_y_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(y_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_y_2022_"}{featureeng}{".csv"}')
    
    #pd.DataFrame(pred_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_pred_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(pred_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_pred_2022_"}{featureeng}{".csv"}')

In [20]:
X_2022

path_abx,usage-Amoxicillin,usage-Amoxicillin/clavulanic acid,usage-Azithromycin,usage-Cefalexin,usage-Cefotaxime,usage-Ceftazidime,usage-Ceftriaxone,usage-Cefuroxime,usage-Ciprofloxacin,usage-Clarithromycin,...,usage-Ofloxacin,usage-Penicillin g,usage-Penicillin v,usage-Piperacillin/tazobactam,usage-Pivmecillinam,usage-Sulfamethoxazole/trimethoprim,usage-Teicoplanin,usage-Tobramycin,usage-Trimethoprim,usage-Vancomycin_iv
fyear,prev1,prev1,prev1,prev1,prev1,prev1,prev1,prev1,prev1,prev1,...,prev6,prev6,prev6,prev6,prev6,prev6,prev6,prev6,prev6,prev6
TrustName.x,,,,,,,,,,,,,,,,,,,,,
trust_1,43.543414,61.542085,74.073751,47.113338,91.755908,86.771975,96.291610,18.634846,17.906112,27.968641,...,37.197471,93.017224,61.258773,48.332335,1.449026,24.724332,28.329524,64.594828,81.328136,74.975693
trust_10,7.024647,95.618305,67.945255,26.135227,38.771409,27.803476,38.020087,60.244065,99.821104,57.412933,...,17.064598,72.336902,97.304887,26.585688,59.166932,51.309300,55.403183,77.515327,62.203375,76.080004
trust_100,89.013059,48.965585,14.687486,56.560720,46.929121,52.639305,49.720410,11.161587,13.600378,56.641477,...,58.814492,68.407108,18.115754,66.714019,55.699463,59.361784,10.368848,51.786997,81.379058,3.858540
trust_11,98.505641,15.747592,72.613709,0.726508,81.422252,51.632737,87.040326,46.012089,31.113268,16.462495,...,28.575755,57.280739,12.946360,71.699512,65.368801,64.203017,40.530252,74.731663,37.376253,86.800343
trust_12,33.115843,7.145848,54.597917,66.811498,14.085004,46.164462,98.994082,29.145219,24.510349,85.668153,...,70.386021,58.249763,0.943332,4.647740,74.564734,70.260087,43.096559,41.566545,81.653227,64.139167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
trust_95,18.426691,93.322743,59.924113,30.228415,67.444832,6.200073,91.877023,93.179899,76.710672,39.436765,...,91.338206,40.686102,82.890742,63.895307,46.488213,8.996386,80.972119,58.545505,40.459161,93.025072
trust_96,28.988106,7.628023,15.663010,20.824715,82.342087,52.056770,8.739968,74.907471,15.719892,73.734181,...,30.659455,3.186763,37.174258,29.398692,20.533038,84.100621,79.072602,98.437329,33.647900,24.834234


In [21]:
pred_2022

array([31.820011, 31.734324, 47.355347, 52.90341 , 46.092472, 48.51985 ,
       41.46591 , 53.976967, 29.776897, 61.907326, 33.241386, 50.483032,
       45.454365, 48.34948 , 50.39512 , 35.93451 , 39.2902  , 40.43824 ,
       46.74315 , 41.59911 , 43.990524, 61.92715 , 36.82423 , 36.524864,
       57.03131 , 33.712616, 44.635246, 47.220966, 35.47853 , 43.98728 ,
       38.865482, 29.41793 , 50.528748, 45.537815, 42.976707, 44.057552,
       60.681038, 42.41594 , 58.888878, 28.770592, 47.028313, 59.72564 ,
       43.090992, 63.62402 , 57.641727, 34.034904, 53.434456, 42.830383,
       41.47614 , 50.31234 , 55.697872, 35.420925, 39.915684, 56.540703,
       62.04401 , 48.8278  , 41.288704, 35.611584, 56.22329 , 55.02983 ,
       49.33842 , 36.242786, 42.122917, 37.296677, 33.724705, 48.846672,
       60.08602 , 33.833973, 36.224804, 62.996708, 50.012653, 40.30278 ,
       37.823044, 41.836056, 65.34069 , 65.01921 , 29.879206, 34.22521 ,
       58.66543 , 50.019398, 58.12687 , 50.7587  , 

In [22]:
# using just the previous year of usage as predictor, without previous resistance
featureeng = "res0usage1defaultparam"
for p in range(22):
    dataset_wide=res_and_usage
    
    dataset_2021_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2020-2021')])
    #d1 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2019-2020']
    d1 = dataset_2021_selabx_nona.iloc[:, np.logical_and(dataset_2021_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2021_selabx_nona.columns.get_level_values(1)=='2019-2020')]
    X_2021 = d1
    y_2021 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns==(path_drug[p], '2020-2021')]

    dataset_2022_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2021-2022')])
    #d1 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2020-2021']
    d1 = dataset_2022_selabx_nona.iloc[:, np.logical_and(dataset_2022_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2022_selabx_nona.columns.get_level_values(1)=='2020-2021')]
    X_2022 = d1
    y_2022 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns==(path_drug[p], '2021-2022')]

    ####
    
    X_2021 = X_2021.rename(columns={'2019-2020': 'prev1'})
    X_2022 = X_2022.rename(columns={'2020-2021': 'prev1'})

    columnsinboth = X_2021.columns.intersection(X_2022.columns) # what is in both
    X_2021 = X_2021[X_2021.columns.intersection(columnsinboth)]
    X_2022 = X_2022[X_2022.columns.intersection(columnsinboth)]

    X_2021_np=np.array(X_2021)
    y_2021_np=np.array(y_2021)
    X_2022_np=np.array(X_2022)
    y_2022_np=np.array(y_2022)

    ####
    model = XGBRegressor(
                         objective='reg:absoluteerror',
                         eval_metric=mean_absolute_error, 
                         seed=42)

    fit = model.fit(
        X_2021_np, 
        y_2021_np, 
        verbose=True)
  
    pred_2021 = fit.predict(X_2021_np) 
    pred_2022 = fit.predict(X_2022_np)
    
    #pd.DataFrame(X_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_X_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(X_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_X_2022_"}{featureeng}{".csv"}')
    
    #pd.DataFrame(y_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_y_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(y_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_y_2022_"}{featureeng}{".csv"}')
    
    #pd.DataFrame(pred_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_pred_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(pred_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_pred_2022_"}{featureeng}{".csv"}')

In [23]:
# using just the previous year of usage as predictor, without previous resistance, but training on 2 years of data as the outcome
featureeng = "res0usage1defaultparam_2input"
for p in range(22):
    dataset_wide=res_and_usage
    
    dataset_2021_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2020-2021')])
    #d1 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2019-2020']
    d1 = dataset_2021_selabx_nona.iloc[:, np.logical_and(dataset_2021_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2021_selabx_nona.columns.get_level_values(1)=='2019-2020')]
    X_2021 = d1
    y_2021 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns==(path_drug[p], '2020-2021')]
    
    dataset_2020_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2019-2020')])
    d1 = dataset_2020_selabx_nona.iloc[:, np.logical_and(dataset_2020_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2020_selabx_nona.columns.get_level_values(1)=='2018-2019')]
    X_2020 = d1
    y_2020 = dataset_2020_selabx_nona.iloc[:, dataset_2020_selabx_nona.columns==(path_drug[p], '2019-2020')]
    
    dataset_2022_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2021-2022')])
    #d1 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2020-2021']
    d1 = dataset_2022_selabx_nona.iloc[:, np.logical_and(dataset_2022_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2022_selabx_nona.columns.get_level_values(1)=='2020-2021')]
    X_2022 = d1
    y_2022 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns==(path_drug[p], '2021-2022')]

    ####
    
    X_2020 = X_2020.rename(columns={'2018-2019': 'prev1'})
    X_2021 = X_2021.rename(columns={'2019-2020': 'prev1'})
    X_2022 = X_2022.rename(columns={'2020-2021': 'prev1'})
    
    y_2020 = y_2020.rename(columns={'2019-2020': 'true'})
    y_2021 = y_2021.rename(columns={'2020-2021': 'true'})
    
    columnsinboth = X_2020.columns.intersection(X_2021.columns) # what is in both
    X_2021 = X_2021[X_2021.columns.intersection(columnsinboth)]
    X_2020 = X_2020[X_2020.columns.intersection(columnsinboth)]

    columnsinboth = X_2021.columns.intersection(X_2022.columns) # what is in both
    X_2021 = X_2021[X_2021.columns.intersection(columnsinboth)]
    X_2022 = X_2022[X_2022.columns.intersection(columnsinboth)]
    
    frames_X = [X_2020, X_2021]
    frames_y = [y_2020, y_2021]
    
    X_train = pd.concat(frames_X)
    y_train = pd.concat(frames_y)
    #
    X_2021_np=np.array(X_train)
    y_2021_np=np.array(y_train)
    X_2022_np=np.array(X_2022)
    y_2022_np=np.array(y_2022)

    ####
    model = XGBRegressor(
                         objective='reg:absoluteerror',
                         eval_metric=mean_absolute_error, 
                         seed=42)

    fit = model.fit(
        X_2021_np, 
        y_2021_np, 
        verbose=True)
  
    pred_2021 = fit.predict(X_2021_np) 
    pred_2022 = fit.predict(X_2022_np)
    
    #pd.DataFrame(X_train).to_csv(f'{path_drug[p].replace("/","_")}{"_X_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(X_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_X_2022_"}{featureeng}{".csv"}')
    
    #pd.DataFrame(y_train).to_csv(f'{path_drug[p].replace("/","_")}{"_y_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(y_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_y_2022_"}{featureeng}{".csv"}')
    
    #pd.DataFrame(pred_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_pred_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(pred_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_pred_2022_"}{featureeng}{".csv"}')

In [24]:
# using just the previous year of usage as predictor, without previous resistance, but training on 3 years of data as the outcome
featureeng = "res0usage1defaultparam_3input"
for p in range(22):
    dataset_wide=res_and_usage
    
    dataset_2021_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2020-2021')])
    #d1 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2019-2020']
    d1 = dataset_2021_selabx_nona.iloc[:, np.logical_and(dataset_2021_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2021_selabx_nona.columns.get_level_values(1)=='2019-2020')]
    X_2021 = d1
    y_2021 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns==(path_drug[p], '2020-2021')]
    
    dataset_2020_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2019-2020')])
    d1 = dataset_2020_selabx_nona.iloc[:, np.logical_and(dataset_2020_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2020_selabx_nona.columns.get_level_values(1)=='2018-2019')]
    X_2020 = d1
    y_2020 = dataset_2020_selabx_nona.iloc[:, dataset_2020_selabx_nona.columns==(path_drug[p], '2019-2020')]
    
    dataset_2019_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2018-2019')])
    d1 = dataset_2019_selabx_nona.iloc[:, np.logical_and(dataset_2019_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2019_selabx_nona.columns.get_level_values(1)=='2017-2018')]
    X_2019 = d1
    y_2019 = dataset_2019_selabx_nona.iloc[:, dataset_2019_selabx_nona.columns==(path_drug[p], '2018-2019')]
    
    dataset_2022_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2021-2022')])
    #d1 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2020-2021']
    d1 = dataset_2022_selabx_nona.iloc[:, np.logical_and(dataset_2022_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2022_selabx_nona.columns.get_level_values(1)=='2020-2021')]
    X_2022 = d1
    y_2022 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns==(path_drug[p], '2021-2022')]

    ####
    X_2019 = X_2019.rename(columns={'2017-2018': 'prev1'})
    X_2020 = X_2020.rename(columns={'2018-2019': 'prev1'})
    X_2021 = X_2021.rename(columns={'2019-2020': 'prev1'})
    X_2022 = X_2022.rename(columns={'2020-2021': 'prev1'})
    
    y_2019 = y_2019.rename(columns={'2018-2019': 'true'})
    y_2020 = y_2020.rename(columns={'2019-2020': 'true'})
    y_2021 = y_2021.rename(columns={'2020-2021': 'true'})
    
    columnsinboth = X_2019.columns.intersection(X_2020.columns) # what is in both
    X_2019 = X_2019[X_2019.columns.intersection(columnsinboth)]
    X_2020 = X_2020[X_2020.columns.intersection(columnsinboth)]

    columnsinboth = X_2020.columns.intersection(X_2021.columns) # what is in both
    X_2021 = X_2021[X_2021.columns.intersection(columnsinboth)]
    X_2020 = X_2020[X_2020.columns.intersection(columnsinboth)]

    columnsinboth = X_2021.columns.intersection(X_2022.columns) # what is in both
    X_2021 = X_2021[X_2021.columns.intersection(columnsinboth)]
    X_2022 = X_2022[X_2022.columns.intersection(columnsinboth)]
    
    frames_X = [X_2019, X_2020, X_2021]
    frames_y = [y_2019, y_2020, y_2021]
    
    X_train = pd.concat(frames_X)
    y_train = pd.concat(frames_y)
    
    X_2021_np=np.array(X_train)
    y_2021_np=np.array(y_train)
    X_2022_np=np.array(X_2022)
    y_2022_np=np.array(y_2022)

    ####
    model = XGBRegressor(
                         objective='reg:absoluteerror',
                         eval_metric=mean_absolute_error, 
                         seed=42)

    fit = model.fit(
        X_2021_np, 
        y_2021_np, 
        verbose=True)
  
    pred_2021 = fit.predict(X_2021_np) 
    pred_2022 = fit.predict(X_2022_np)
    
    #pd.DataFrame(X_train).to_csv(f'{path_drug[p].replace("/","_")}{"_X_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(X_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_X_2022_"}{featureeng}{".csv"}')
    
    #pd.DataFrame(y_train).to_csv(f'{path_drug[p].replace("/","_")}{"_y_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(y_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_y_2022_"}{featureeng}{".csv"}')
    
    #pd.DataFrame(pred_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_pred_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(pred_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_pred_2022_"}{featureeng}{".csv"}')

In [25]:
X_train

path_abx,usage-Amoxicillin,usage-Amoxicillin/clavulanic acid,usage-Azithromycin,usage-Cefalexin,usage-Cefotaxime,usage-Ceftazidime,usage-Ceftriaxone,usage-Cefuroxime,usage-Ciprofloxacin,usage-Clarithromycin,...,usage-Ofloxacin,usage-Penicillin g,usage-Penicillin v,usage-Piperacillin/tazobactam,usage-Pivmecillinam,usage-Sulfamethoxazole/trimethoprim,usage-Teicoplanin,usage-Tobramycin,usage-Trimethoprim,usage-Vancomycin_iv
fyear,prev1,prev1,prev1,prev1,prev1,prev1,prev1,prev1,prev1,prev1,...,prev1,prev1,prev1,prev1,prev1,prev1,prev1,prev1,prev1,prev1
TrustName.x,,,,,,,,,,,,,,,,,,,,,
trust_1,63.201193,26.285770,88.252980,17.082419,61.831280,53.874495,66.721736,33.437953,65.078278,98.820157,...,40.659269,14.657329,9.086591,50.920616,62.191665,1.480838,2.057991,38.195667,4.488732,80.125160
trust_10,65.550189,52.758933,64.255252,76.089867,34.472539,66.731731,73.797272,70.658393,88.986892,43.355568,...,18.457713,90.781540,67.091870,93.724138,20.220527,29.888399,30.734600,43.709714,12.690702,27.904875
trust_100,64.739552,31.808809,88.649359,22.188428,63.620649,82.216285,5.192691,0.887584,62.297478,53.805948,...,40.204959,6.377259,18.165048,37.616057,93.107279,43.972962,90.850595,35.329115,56.947604,48.952738
trust_11,14.047818,58.889535,22.677714,70.239422,24.131469,8.414782,31.111570,88.991246,3.370828,78.437888,...,54.053345,48.859185,0.172448,55.141637,9.005925,81.206227,76.906756,17.646400,27.077263,1.241273
trust_12,9.768865,47.216570,0.412042,18.955929,87.684516,65.629573,44.463442,2.404271,95.809603,26.840692,...,30.653536,33.125505,51.209062,61.855701,44.103812,57.577503,75.803931,87.442478,81.902848,41.391283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
trust_95,93.305885,28.333360,46.405956,24.808293,65.465171,68.102180,22.829996,31.662838,72.751239,69.755588,...,42.989577,75.131294,25.509623,97.924190,92.686623,36.539709,76.053099,11.157303,31.848887,53.875282
trust_96,93.648620,77.166254,76.604359,58.902885,28.405149,72.011746,13.200383,42.855401,20.946196,77.692996,...,87.309616,46.902919,15.606247,90.788710,44.717936,37.109826,14.645040,61.403303,58.993695,83.661411


In [26]:
y_2022_np = y_2022_np.flatten()

In [27]:
vals= np.abs(shap_values[list(list(np.where(abs(np.subtract(y_2022_np, pred_2022))>10))[0])]).mean(0) 
vals.shape

(292,)

In [28]:
vals= np.abs(shap_values).mean(0) 
vals

array([0.26002085, 0.1731267 , 0.20019965, 0.07687942, 0.11218303,
       0.11675388, 0.43109548, 0.42670265, 0.05285409, 0.15486652,
       0.03451036, 0.26487735, 0.06789791, 0.07570087, 0.08739043,
       0.0266067 , 0.05085838, 0.2506279 , 0.26958987, 0.02520965,
       0.13962528, 0.06490181, 0.05668904, 0.6152267 , 0.0497711 ,
       0.22118062, 0.02218591, 0.03710191, 0.03583547, 0.16498403,
       0.01990631, 0.18475643, 0.04148802, 0.1628368 , 0.03927661,
       0.24290262, 0.02301532, 0.05411195, 2.6834738 , 0.11165591,
       0.08658719, 0.866647  , 0.        , 0.09988587, 0.07667752,
       0.0784461 , 0.19755627, 0.01805132, 0.        , 0.02587702,
       0.        , 0.20174503, 0.        , 0.00770843, 0.0992168 ,
       0.01664726, 0.10783334, 0.0273951 , 0.02340336, 0.31017235,
       0.2485092 , 0.03268421, 0.07577798, 0.14729397, 0.1790106 ,
       0.        , 0.07796347, 0.891138  , 0.        , 0.1253927 ,
       0.0799414 , 0.        , 0.07531876, 0.03569414, 0.     

In [29]:
vals= np.abs(shap_values[list(list(np.where(abs(np.subtract(y_2022_np, pred_2022))>10))[0])]).mean(0) 
feature_importance = pd.DataFrame(list(zip(X_2022.columns,vals)),columns=['col_name','feature_importance_vals']) 
feature_importance.sort_values(by=['feature_importance_vals'],ascending=False,inplace=True) 
print(feature_importance.head())

                         col_name  feature_importance_vals
23  (usage-Nitrofurantoin, prev1)                 0.625692
7       (usage-Cefuroxime, prev1)                 0.458961
6      (usage-Ceftriaxone, prev1)                 0.438735
11        (usage-Colistin, prev1)                 0.283185
17    (usage-Levofloxacin, prev1)                 0.265397


In [30]:
# keeping only those features that have a mean Shap value in the training dataset greater than that of white noise
featureeng = "res4usage6defaultparam_shapfs"
for p in range(22):

    dataset_wide=res_and_usage
    
    dataset_2021_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2020-2021')])
    d1 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2019-2020']
    d2 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2018-2019']
    d3 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2017-2018']
    d4 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2016-2017']
    d5 = dataset_2021_selabx_nona.iloc[:, np.logical_and(dataset_2021_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2021_selabx_nona.columns.get_level_values(1)=='2015-2016')]
    d6 = dataset_2021_selabx_nona.iloc[:, np.logical_and(dataset_2021_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2021_selabx_nona.columns.get_level_values(1)=='2014-2015')]
    X_2021 = pd.merge(d1,d2,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d3,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d4,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d5,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d6,left_index=True,right_index=True)
    X_2021["noise"] = np.random.normal(loc = 0, scale = 1, size = X_2021.shape[0])
    y_2021 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns==(path_drug[p], '2020-2021')]

    dataset_2022_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2021-2022')])
    d1 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2020-2021']
    d2 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2019-2020']
    d3 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2018-2019']
    d4 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2017-2018']
    d5 = dataset_2022_selabx_nona.iloc[:, np.logical_and(dataset_2022_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2022_selabx_nona.columns.get_level_values(1)=='2016-2017')]
    d6 = dataset_2022_selabx_nona.iloc[:, np.logical_and(dataset_2022_selabx_nona.columns.get_level_values(0).str.contains(pat = 'usage'), 
                                                     dataset_2022_selabx_nona.columns.get_level_values(1)=='2015-2016')]
    X_2022 = pd.merge(d1,d2,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d3,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d4,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d5,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d6,left_index=True,right_index=True)
    X_2022["noise"] = np.random.normal(loc = 0, scale = 1, size = X_2022.shape[0])
    y_2022 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns==(path_drug[p], '2021-2022')]

    ####
    
    X_2021 = X_2021.rename(columns={'2014-2015': 'prev6', '2015-2016': 'prev5', '2016-2017': 'prev4', '2017-2018': 'prev3', '2018-2019': 'prev2', '2019-2020': 'prev1'})
    X_2022 = X_2022.rename(columns={'2015-2016': 'prev6', '2016-2017': 'prev5', '2017-2018': 'prev4', '2018-2019': 'prev3', '2019-2020': 'prev2', '2020-2021': 'prev1'})

    columnsinboth = X_2021.columns.intersection(X_2022.columns) # what is in both
    X_2021 = X_2021[X_2021.columns.intersection(columnsinboth)]
    X_2022 = X_2022[X_2022.columns.intersection(columnsinboth)]

    X_2021_np=np.array(X_2021)
    y_2021_np=np.array(y_2021)
    X_2022_np=np.array(X_2022)
    y_2022_np=np.array(y_2022)

    ####
    model = XGBRegressor(
                         objective='reg:absoluteerror',
                         eval_metric=mean_absolute_error, 
                         seed=42)

    fit = model.fit(
        X_2021_np, 
        y_2021_np, 
        verbose=True)
 
    pred_2021 = fit.predict(X_2021_np) 
    pred_2022 = fit.predict(X_2022_np)
    
    print(path_drug[p])
    # DF, based on which importance is checked
    X_importance = X_2021_np
    # Explain model predictions using shap library:
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_importance)
    # Plot summary_plot as barplot:
    #shap.summary_plot(shap_values, X_importance, plot_type='bar')
    vals= np.abs(shap_values).mean(0) 
    feature_importance = pd.DataFrame(list(zip(X_2021.columns,vals)),columns=['col_name','feature_importance_vals']) 
    feature_importance.sort_values(by=['feature_importance_vals'],ascending=False,inplace=True) 
    #print(feature_importance.head())
    X_2021_fs = X_2021[X_2021.columns.intersection(list(feature_importance[0:feature_importance.index[feature_importance['col_name'] == ('noise', '')][0]]['col_name']))]
    X_2022_fs = X_2022[X_2022.columns.intersection(list(feature_importance[0:feature_importance.index[feature_importance['col_name'] == ('noise', '')][0]]['col_name']))]
    X_2021_np=np.array(X_2021_fs)
    X_2022_np=np.array(X_2022_fs)
 
    ####
    model = XGBRegressor(
                         objective='reg:absoluteerror',
                         eval_metric=mean_absolute_error, 
                         seed=42)

    fit = model.fit(
        X_2021_np, 
        y_2021_np, 
        verbose=True)
 
    pred_2021 = fit.predict(X_2021_np) 
    pred_2022 = fit.predict(X_2022_np)
    #pd.DataFrame(X_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_X_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(X_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_X_2022_"}{featureeng}{".csv"}')
    
    #pd.DataFrame(y_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_y_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(y_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_y_2022_"}{featureeng}{".csv"}')
    
    #pd.DataFrame(pred_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_pred_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(pred_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_pred_2022_"}{featureeng}{".csv"}')

E. COLI_AMOXICILLIN/CLAVULANIC ACID-res


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


E. COLI_CARBAPENEMS-res


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


E. COLI_CEPHALOSPORINS-res


...

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


P. AERUGINOSA_GENTAMICIN-res


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


P. AERUGINOSA_PIPERACILLIN/TAZOBACTAM-res


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


In [31]:
# using one year of historical usage and resistance, but 3 years as outcome
featureeng = "res1usage1_defaultparam_3d"
for p in range(22):
    dataset_wide=res_and_usage
    
    dataset_2021_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2020-2021')])
    d1 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2019-2020']
    X_2021 = d1
    y_2021 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns==(path_drug[p], '2020-2021')]
    
    dataset_2020_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2019-2020')])
    d1 = dataset_2020_selabx_nona.iloc[:, dataset_2020_selabx_nona.columns.get_level_values(1)=='2018-2019']
    X_2020 = d1
    y_2020 = dataset_2020_selabx_nona.iloc[:, dataset_2020_selabx_nona.columns==(path_drug[p], '2019-2020')]
    
    dataset_2019_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2018-2019')])
    d1 = dataset_2019_selabx_nona.iloc[:, dataset_2019_selabx_nona.columns.get_level_values(1)=='2017-2018']
    X_2019 = d1
    y_2019 = dataset_2019_selabx_nona.iloc[:, dataset_2019_selabx_nona.columns==(path_drug[p], '2018-2019')]
    
    dataset_2022_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2021-2022')])
    d1 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2020-2021']
    X_2022 = d1
    y_2022 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns==(path_drug[p], '2021-2022')]

    ####
    X_2019 = X_2019.rename(columns={'2017-2018': 'prev1'})
    X_2020 = X_2020.rename(columns={'2018-2019': 'prev1'})
    X_2021 = X_2021.rename(columns={'2019-2020': 'prev1'})
    X_2022 = X_2022.rename(columns={'2020-2021': 'prev1'})
    
    y_2019 = y_2019.rename(columns={'2018-2019': 'true'})
    y_2020 = y_2020.rename(columns={'2019-2020': 'true'})
    y_2021 = y_2021.rename(columns={'2020-2021': 'true'})
    
    columnsinboth = X_2019.columns.intersection(X_2020.columns) # what is in both
    X_2019 = X_2019[X_2019.columns.intersection(columnsinboth)]
    X_2020 = X_2020[X_2020.columns.intersection(columnsinboth)]

    columnsinboth = X_2020.columns.intersection(X_2021.columns) # what is in both
    X_2021 = X_2021[X_2021.columns.intersection(columnsinboth)]
    X_2020 = X_2020[X_2020.columns.intersection(columnsinboth)]

    columnsinboth = X_2021.columns.intersection(X_2022.columns) # what is in both
    X_2021 = X_2021[X_2021.columns.intersection(columnsinboth)]
    X_2022 = X_2022[X_2022.columns.intersection(columnsinboth)]
    
    frames_X = [X_2019, X_2020, X_2021]
    frames_y = [y_2019, y_2020, y_2021]
    
    X_train = pd.concat(frames_X)
    y_train = pd.concat(frames_y)
    
    X_2021_np=np.array(X_train)
    y_2021_np=np.array(y_train)
    X_2022_np=np.array(X_2022)
    y_2022_np=np.array(y_2022)

    ####
    model = XGBRegressor(
                         objective='reg:absoluteerror',
                         eval_metric=mean_absolute_error, 
                         seed=42)

    fit = model.fit(
        X_2021_np, 
        y_2021_np, 
        verbose=True)
  
    pred_2021 = fit.predict(X_2021_np) 
    pred_2022 = fit.predict(X_2022_np)
    
    #pd.DataFrame(X_train).to_csv(f'{path_drug[p].replace("/","_")}{"_X_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(X_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_X_2022_"}{featureeng}{".csv"}')
    
    #pd.DataFrame(y_train).to_csv(f'{path_drug[p].replace("/","_")}{"_y_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(y_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_y_2022_"}{featureeng}{".csv"}')
    
    #pd.DataFrame(pred_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_pred_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(pred_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_pred_2022_"}{featureeng}{".csv"}')

In [32]:
# using 2 years of historical usage and resistance, but 2 years as outcome
featureeng = "res2usage2_defaultparam_2d"
for p in range(22):
    dataset_wide=res_and_usage
    
    dataset_2021_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2020-2021')])
    d1 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2019-2020']
    d2 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2018-2019']
    X_2021 = pd.merge(d1,d2,left_index=True,right_index=True)
    y_2021 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns==(path_drug[p], '2020-2021')]
    
    dataset_2020_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2019-2020')])
    d1 = dataset_2020_selabx_nona.iloc[:, dataset_2020_selabx_nona.columns.get_level_values(1)=='2018-2019']
    d2 = dataset_2020_selabx_nona.iloc[:, dataset_2020_selabx_nona.columns.get_level_values(1)=='2017-2018']
    X_2020 = pd.merge(d1,d2,left_index=True,right_index=True)
    y_2020 = dataset_2020_selabx_nona.iloc[:, dataset_2020_selabx_nona.columns==(path_drug[p], '2019-2020')]
    
    dataset_2022_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2021-2022')])
    d1 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2020-2021']
    d2 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2019-2020']
    X_2022 = pd.merge(d1,d2,left_index=True,right_index=True)
    y_2022 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns==(path_drug[p], '2021-2022')]

    ####
    X_2020 = X_2020.rename(columns={'2018-2019': 'prev1', '2017-2018': 'prev2'})
    X_2021 = X_2021.rename(columns={'2019-2020': 'prev1', '2018-2019': 'prev2'})
    X_2022 = X_2022.rename(columns={'2020-2021': 'prev1', '2019-2020': 'prev2'})
    
    y_2020 = y_2020.rename(columns={'2019-2020': 'true'})
    y_2021 = y_2021.rename(columns={'2020-2021': 'true'})
    
    columnsinboth = X_2020.columns.intersection(X_2021.columns) # what is in both
    X_2021 = X_2021[X_2021.columns.intersection(columnsinboth)]
    X_2020 = X_2020[X_2020.columns.intersection(columnsinboth)]

    columnsinboth = X_2021.columns.intersection(X_2022.columns) # what is in both
    X_2021 = X_2021[X_2021.columns.intersection(columnsinboth)]
    X_2022 = X_2022[X_2022.columns.intersection(columnsinboth)]
    
    frames_X = [X_2020, X_2021]
    frames_y = [y_2020, y_2021]
    
    X_train = pd.concat(frames_X)
    y_train = pd.concat(frames_y)
    
    X_2021_np=np.array(X_train)
    y_2021_np=np.array(y_train)
    X_2022_np=np.array(X_2022)
    y_2022_np=np.array(y_2022)

    ####
    model = XGBRegressor(
                         objective='reg:absoluteerror',
                         eval_metric=mean_absolute_error, 
                         seed=42)

    fit = model.fit(
        X_2021_np, 
        y_2021_np, 
        verbose=True)
  
    pred_2021 = fit.predict(X_2021_np) 
    pred_2022 = fit.predict(X_2022_np)
    
    #pd.DataFrame(X_train).to_csv(f'{path_drug[p].replace("/","_")}{"_X_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(X_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_X_2022_"}{featureeng}{".csv"}')
    
    #pd.DataFrame(y_train).to_csv(f'{path_drug[p].replace("/","_")}{"_y_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(y_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_y_2022_"}{featureeng}{".csv"}')
    
    #pd.DataFrame(pred_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_pred_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(pred_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_pred_2022_"}{featureeng}{".csv"}')

In [33]:
# using 3 years of historical usage and resistance, but 1 year as outcome
featureeng = "res3usage3_defaultparam_1d"
for p in range(22):
    dataset_wide=res_and_usage
    
    dataset_2021_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2020-2021')])
    d1 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2019-2020']
    d2 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2018-2019']
    d3 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2017-2018']
    
    X_2021 = pd.merge(d1,d2,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d3,left_index=True,right_index=True)
    
    y_2021 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns==(path_drug[p], '2020-2021')]
    
    dataset_2022_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2021-2022')])
    d1 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2020-2021']
    d2 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2019-2020']
    d3 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2018-2019']
    
    X_2022 = pd.merge(d1,d2,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d3,left_index=True,right_index=True)
    y_2022 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns==(path_drug[p], '2021-2022')]

    ####

    X_2021 = X_2021.rename(columns={'2019-2020': 'prev1', '2018-2019': 'prev2', '2017-2018': 'prev3'})
    X_2022 = X_2022.rename(columns={'2020-2021': 'prev1', '2019-2020': 'prev2', '2018-2019': 'prev3'})
    
    columnsinboth = X_2021.columns.intersection(X_2022.columns) # what is in both
    X_2021 = X_2021[X_2021.columns.intersection(columnsinboth)]
    X_2022 = X_2022[X_2022.columns.intersection(columnsinboth)]
    
    X_2021_np=np.array(X_2021)
    y_2021_np=np.array(y_2021)
    X_2022_np=np.array(X_2022)
    y_2022_np=np.array(y_2022)

    ####
    model = XGBRegressor(
                         objective='reg:absoluteerror',
                         eval_metric=mean_absolute_error, 
                         seed=42)

    fit = model.fit(
        X_2021_np, 
        y_2021_np, 
        verbose=True)
  
    pred_2021 = fit.predict(X_2021_np) 
    pred_2022 = fit.predict(X_2022_np)
    
    #pd.DataFrame(X_train).to_csv(f'{path_drug[p].replace("/","_")}{"_X_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(X_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_X_2022_"}{featureeng}{".csv"}')
    
    #pd.DataFrame(y_train).to_csv(f'{path_drug[p].replace("/","_")}{"_y_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(y_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_y_2022_"}{featureeng}{".csv"}')
    
    #pd.DataFrame(pred_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_pred_2021_"}{featureeng}{".csv"}')
    #pd.DataFrame(pred_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_pred_2022_"}{featureeng}{".csv"}')

In [34]:
allpath_drug = path_drug
matchers = ['E. COLI','MSSA']
matching = [s for s in allpath_drug if any(allpath_drug in s for allpath_drug in matchers)]
matching
N = len(matching)
N
for p in range(N):
    print(matching[p])
path_drug=matching

E. COLI_AMOXICILLIN/CLAVULANIC ACID-res
E. COLI_CARBAPENEMS-res
E. COLI_CEPHALOSPORINS-res
E. COLI_CIPROFLOXACIN-res
E. COLI_GENTAMICIN-res
E. COLI_PIPERACILLIN/TAZOBACTAM-res
MSSA_CLARITHROMYCIN-res
MSSA_CLINDAMYCIN-res
MSSA_ERYTHROMYCIN-res
MSSA_TETRACYCLINE-res
MSSA_VANCOMYCIN-res


In [35]:
# using the values in 2016-2017 and consecutive differences from there on as features (rather than actual values for all the years)
for p in range(N):
    dataset_wide=res_and_usage
    
    dataset_2021_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2020-2021')])
    d1 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2019-2020_diff']
    d2 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2018-2019_diff']
    d3 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2017-2018_diff']
    d4 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2016-2017']
    X_2021 = pd.merge(d1,d2,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d3,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d4,left_index=True,right_index=True)
    y_2021 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns==(path_drug[p], '2020-2021')]

    dataset_2022_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2021-2022')])
    d1 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2020-2021_diff']
    d2 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2019-2020_diff']
    d3 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2018-2019_diff']
    d4 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2017-2018']
    X_2022 = pd.merge(d1,d2,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d3,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d4,left_index=True,right_index=True)
    y_2022 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns==(path_drug[p], '2021-2022')]

    ####
    
    X_2021 = X_2021.rename(columns={'2016-2017': 'prev4', '2017-2018_diff': 'prev3', '2018-2019_diff': 'prev2', '2019-2020_diff': 'prev1'})
    X_2022 = X_2022.rename(columns={'2017-2018': 'prev4', '2018-2019_diff': 'prev3', '2019-2020_diff': 'prev2', '2020-2021_diff': 'prev1'})

    columnsinboth = X_2021.columns.intersection(X_2022.columns) # what is in both
    X_2021 = X_2021[X_2021.columns.intersection(columnsinboth)]
    X_2022 = X_2022[X_2022.columns.intersection(columnsinboth)]

    X_2021_np=np.array(X_2021)
    y_2021_np=np.array(y_2021)
    X_2022_np=np.array(X_2022)
    y_2022_np=np.array(y_2022)

    ####
    model = XGBRegressor(
                         objective='reg:absoluteerror',
                         eval_metric=mean_absolute_error, 
                         seed=42)

    fit = model.fit(
        X_2021_np, 
        y_2021_np, 
        verbose=True)

    pred_2021 = fit.predict(X_2021_np) 
    pred_2022 = fit.predict(X_2022_np)
    
    #pd.DataFrame(X_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_X_2021_3diffandtrue_clean"}{".csv"}')
    #pd.DataFrame(X_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_X_2022_3diffandtrue_clean"}{".csv"}')
    
    #pd.DataFrame(y_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_y_2021_3diffandtrue_clean"}{".csv"}')
    #pd.DataFrame(y_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_y_2022_3diffandtrue_clean"}{".csv"}')
    
    #pd.DataFrame(pred_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_pred_2021"}{"_defaultparam_regabserr_3diffandtrue_clean.csv"}')
    #pd.DataFrame(pred_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_pred_2022"}{"_defaultparam_regabserr_3diffandtrue_clean.csv"}')

In [36]:
matchers = ['P. AERUGINOSA','KLEBSIELLA SP.']
matching = [s for s in allpath_drug if any(allpath_drug in s for allpath_drug in matchers)]
matching
N = len(matching)
N
for p in range(N):
    print(matching[p])
path_drug=matching

KLEBSIELLA SP._AMOXICILLIN/CLAVULANIC ACID-res
KLEBSIELLA SP._CARBAPENEMS-res
KLEBSIELLA SP._CEPHALOSPORINS-res
KLEBSIELLA SP._CIPROFLOXACIN-res
KLEBSIELLA SP._GENTAMICIN-res
KLEBSIELLA SP._PIPERACILLIN/TAZOBACTAM-res
P. AERUGINOSA_CARBAPENEMS-res
P. AERUGINOSA_CEFTAZIDIME-res
P. AERUGINOSA_CIPROFLOXACIN-res
P. AERUGINOSA_GENTAMICIN-res
P. AERUGINOSA_PIPERACILLIN/TAZOBACTAM-res


In [37]:
# using the values in 2017-2018 and consecutive differences from there on as features (rather than actual values for all the years)
# data is only available from 2017-2018 onwards
for p in range(N):
    dataset_wide=res_and_usage
    
    dataset_2021_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2020-2021')])
    d1 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2019-2020_diff']
    d2 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2018-2019_diff']
    d3 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns.get_level_values(1)=='2017-2018']
    X_2021 = pd.merge(d1,d2,left_index=True,right_index=True)
    X_2021 = pd.merge(X_2021,d3,left_index=True,right_index=True)
    y_2021 = dataset_2021_selabx_nona.iloc[:, dataset_2021_selabx_nona.columns==(path_drug[p], '2020-2021')]

    dataset_2022_selabx_nona=dataset_wide.dropna(subset=[(path_drug[p], '2021-2022')])
    d1 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2020-2021_diff']
    d2 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2019-2020_diff']
    d3 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns.get_level_values(1)=='2018-2019']
    X_2022 = pd.merge(d1,d2,left_index=True,right_index=True)
    X_2022 = pd.merge(X_2022,d3,left_index=True,right_index=True)
    y_2022 = dataset_2022_selabx_nona.iloc[:, dataset_2022_selabx_nona.columns==(path_drug[p], '2021-2022')]

    ####
    
    X_2021 = X_2021.rename(columns={'2017-2018': 'prev3', '2018-2019_diff': 'prev2', '2019-2020_diff': 'prev1'})
    X_2022 = X_2022.rename(columns={'2018-2019': 'prev3', '2019-2020_diff': 'prev2', '2020-2021_diff': 'prev1'})

    columnsinboth = X_2021.columns.intersection(X_2022.columns) # what is in both
    X_2021 = X_2021[X_2021.columns.intersection(columnsinboth)]
    X_2022 = X_2022[X_2022.columns.intersection(columnsinboth)]

    X_2021_np=np.array(X_2021)
    y_2021_np=np.array(y_2021)
    X_2022_np=np.array(X_2022)
    y_2022_np=np.array(y_2022)

    ####
    model = XGBRegressor(
                         objective='reg:absoluteerror',
                         eval_metric=mean_absolute_error, 
                         seed=42)

    fit = model.fit(
        X_2021_np, 
        y_2021_np, 
        verbose=True)

    pred_2021 = fit.predict(X_2021_np) 
    pred_2022 = fit.predict(X_2022_np)
    
    #pd.DataFrame(X_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_X_2021_3diffandtrue_clean"}{".csv"}')
    #pd.DataFrame(X_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_X_2022_3diffandtrue_clean"}{".csv"}')
    
    #pd.DataFrame(y_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_y_2021_3diffandtrue_clean"}{".csv"}')
    #pd.DataFrame(y_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_y_2022_3diffandtrue_clean"}{".csv"}')
    
    #pd.DataFrame(pred_2021).to_csv(f'{path_drug[p].replace("/","_")}{"_pred_2021"}{"_defaultparam_regabserr_3diffandtrue_clean.csv"}')
    #pd.DataFrame(pred_2022).to_csv(f'{path_drug[p].replace("/","_")}{"_pred_2022"}{"_defaultparam_regabserr_3diffandtrue_clean.csv"}')